**PURPOSE**: To create Simulated Lightcurve Data

**GOALS**:
1. Use the fitted distributions developed in EDA_ZERO.ipynb to create 10,000 Synthetic lightcurves using the BATMAN package:
2. Use the TSFRESH library to implement Feature Extraction for all these lightcurves.


**INPUTS:**
1. Statistical Parameters of Distributions which were fitted to important Transit Parameters (Orbital Period, Transit Duration) and Stellar Parameters (Orbital Radius) . Obtained from EDA_ZERO.ipynb

**OUTPUTS**:
1. 10,000 Simulated Exoplanet Light Curves as individual CSV files
2. CSV File containing Features extracted for each of these using the TSFRESH Library



In [ ]:
###################################################################################
# Created Simulated Lightcurves
###################################################################################
!pip install lightkurve astroquery pandas numpy tqdm  xgboost
!bash pip install --upgrade astroquery
#!pip install tsfel
!pip install stumpy
!pip install tsfresh
!pip install batman-package


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 102.3 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=528d4177f1f20642438817b3f2b634bd28cb137521c5c05347c0c1233e595fa7
  Stored in directory: /root/.cache/pip/wheels/71/ae/f7/5b7e1e16a133d094431428169e9a8d9f750afe5459f0333c9b
  Created wheel for memoization:

In [ ]:
import os
import pandas as pd
import numpy as np
import batman
import tsfresh
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters
#import tsfel
#from tsfel import time_series_features_extractor, get_features_by_domain
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from pathlib import Path
#import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import stumpy

# Known Scientific constants G = Gravitational Constant, M_sun = Mass of the Sun, R_sun = Radius of the Sun
from astropy.constants import G, M_sun, R_sun

#import lightkurve as lk
#from lightkurve import LightCurve



In [ ]:
#Define paths and constants

ROOT  = Path("/content/drive/MyDrive/Berkeley_AIML/Capstone/lightcurves")
#EXOPLANET_INPUT_CURVES_PATH = ROOT / "chosen_exoplanet_curves"
SIMULATED_CURVES_PATH       = ROOT / "simcurves2"
ANCILLARY                   = ROOT / "ancillary"
OUTPUT_PATH                 = ROOT / "orbital_prediction_simcurve_XGB_model1"

# Kepler long cadence parameters for number of observations per quarter
CADENCE_MIN = 29.4  # minutes
CADENCE_SEC = CADENCE_MIN * 60  # seconds
DAYS_PER_QUARTER = 90
POINTS_PER_QUARTER = int(DAYS_PER_QUARTER * 24 * 60 / CADENCE_MIN)  # ~4,416
NUM_QUARTERS = 4
TOTAL_POINTS = POINTS_PER_QUARTER * NUM_QUARTERS  # ~17,664
# Array of time values for 4 quarters of 90 days each at a cadence of 29.4 minutes approx.
TIME_ARRAY = np.linspace(0, DAYS_PER_QUARTER * NUM_QUARTERS, TOTAL_POINTS)


# Parameters of Log-normal distribution of ORBITAL PERIOD (from EDA_ZERO.ipynb) fitted to known Kepler long cadence lightcurves with exoplanets
ORB_PERIOD_LOGNORM_SHAPE = 1.3330          # sigma
ORB_PERIOD_LOGNORM_LOC = 0.0               # location, must be 0
ORB_PERIOD_LOGNORM_SCALE = 11.5893         # exp(mu)

# Parameters of Log-normal distribution of STELLAR RADII (from EDA_ZERO.ipynb) fitted to known Kepler long cadence lightcurves with exoplanets
ST_RAD_LOGNORM_SHAPE = 0.298937          # sigma
ST_RAD_LOGNORM_LOC = 0.0               # location, must be 0
ST_RAD_LOGNORM_SCALE = 0.961700         # exp(mu)

# Parameters of Normal distribution of STELLAR MASSES (from EDA_ZERO.ipynb) fitted to known Kepler long cadence lightcurves with exoplanets
ST_MASS_LOC = 0.963588   # mu
ST_MASS_SCALE = 0.193144 # sigma

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(SIMULATED_CURVES_PATH, exist_ok=True)

In [ ]:

#EXPECTED_OBS = {"Q4": 4400, "Q5": 4410, "Q6": 4460, "Q7": 4425}


In [ ]:
# Generate Sample Inputs to BATMAN based on Statistical Parameters of distributions fitted to Transit and Stellar Parameters.
def sample_period():
    return lognorm.rvs(s=ORB_PERIOD_LOGNORM_SHAPE, loc=ORB_PERIOD_LOGNORM_LOC, scale=LOGNORM_SCALE)

def sample_stellar_mass():
    return np.random.normal(ST_MASS_LOC, ST_MASS_SCALE)

def sample_stellar_radius():
    return lognorm.rvs(s=ST_RAD_LOGNORM_SHAPE, loc=ST_RAD_LOGNORM_LOC, scale=ST_RAD_LOGNORM_SCALE)

def compute_a_over_rstar(P_days, M_star_solar, R_star_solar):
    P_sec = P_days * 86400
    M_star = M_star_solar * M_sun.value
    R_star = R_star_solar * R_sun.value
    a_meters = ((G.value * M_star * P_sec ** 2) / (4 * np.pi ** 2)) ** (1 / 3)
    return a_meters / R_star

def sample_inclination_transiting(a_over_rstar):
    # Ensure b = (a/R_*) * cos(i) < 1 for transit
    max_cos_i = min(1.0 / a_over_rstar, 1.0)
    max_inc_deg = np.degrees(np.arccos(max_cos_i))
    return np.random.uniform(max_inc_deg, 90.0)

In [ ]:

def generate_lightcurve(planet_name, noise_level=0.0001):
    # 1. Sample orbital period
    period = sample_period()
    M_star = sample_stellar_mass()   # in solar masses
    R_star = sample_stellar_mass()  # in solar radii

    # 3. Compute semi-major axis in stellar radii
    a_over_rstar = compute_a_over_rstar(period, M_star, R_star)

    # 4. Sample inclination (transiting geometry)
    inc = sample_inclination_transiting(a_over_rstar)

    # 5. Sample planet radius (Rp/R*)
    rp = np.random.uniform(0.01, 0.15)

    # 6. Sample transit mid-time
    t0 = np.random.uniform(0, period)

    # Set up transit parameters
    params = batman.TransitParams()
    params.t0 = t0
    params.per = period
    params.rp = rp
    params.a = a_over_rstar
    params.inc = inc
    params.ecc = 0.0
    params.w = 90.0
    params.limb_dark = "quadratic"
    params.u = [0.1, 0.3]

    # Generate model
    m = batman.TransitModel(params, TIME_ARRAY)
    flux = m.light_curve(params)
    flux += np.random.normal(0, noise_level, len(TIME_ARRAY))

    # Return as DataFrame
    return pd.DataFrame({
        "time": TIME_ARRAY,
        "flux": flux
       # "planet_name": planet_name,
       # "period": period,
       # "a_over_rstar": a_over_rstar,
       # "inc": inc,
       # "rp": rp,
       # "t0": t0,
       # "M_star": M_star,
       # "R_star": R_star
    }), period, rp, a_over_rstar, inc, t0

In [ ]:

# Generate 10,000 lightcurves
# Suppress TSFresh warnings
import logging
from scipy.stats import lognorm
logging.getLogger('tsfresh.feature_extraction.settings').setLevel(logging.ERROR)
n_simulations = 10000
batch_size = 200

simulated_params = []
features = []

simulated_params_filename = os.path.join(OUTPUT_PATH, "simcurves_fitted_params2.csv")
simulated_curve_features_filename = os.path.join(OUTPUT_PATH, "simcurves_fitted_features2.csv")
print(simulated_params_filename)
print(simulated_curve_features_filename)

# Ensure fresh output files
if os.path.exists(simulated_params_filename):
    os.remove(simulated_params_filename)
if os.path.exists(simulated_curve_features_filename):
    os.remove(simulated_curve_features_filename)
print("Generating lightcurves...")
for i in range(n_simulations):
    planet_name = f"SimPlanet_{i+1}"
    #period = np.random.uniform(0.5, 250)
    #rp = np.random.uniform(0.01, 0.15)
    #period = lognorm.rvs(s=LOGNORM_SHAPE, loc=LOGNORM_LOC, scale=LOGNORM_SCALE)
    #a = np.random.uniform(5, 50)
    #inc = np.random.uniform(85, 90)
    #t0 = np.random.uniform(0, period)
    noise_level = np.random.uniform(0.00005, 0.0002)

    # Generate and store lightcurve
    lc_df, period, rp, a, inc, t0 = generate_lightcurve(planet_name, noise_level)
    #lc_df = generate_lightcurve(planet_name, period, rp, a, inc, t0, noise_level)
    #lc_df.to_csv(os.path.join(SIMULATED_CURVES_PATH, f"{planet_name}_simcurve.csv"), index=False)

    simulated_params.append({
        "planet_name": planet_name,
        "period": period,
        "rp": rp,
        "a": a,
        "inc": inc,
        "t0": t0,
        "noise_level": noise_level
    })

    if 'flux' in lc_df.columns:
        lc_df = lc_df.rename(columns={'flux': 'value'})

    lc_df['planet_name'] = planet_name  # required for column_id
    feats = extract_features(
        lc_df,
        column_id='planet_name',
        column_sort='time',
        column_value='value',
        default_fc_parameters=EfficientFCParameters(),
        disable_progressbar=True,
        show_warnings=False,
        n_jobs=0
    )
    feats.index.name = 'planet_name'
    features.append(feats)

    # Process and save batch
    if (i + 1) % batch_size == 0 or (i + 1) == n_simulations:
        print(f"Processing batch ending at simulation {i + 1}")

        params_df = pd.DataFrame(simulated_params)
        features_df = pd.concat(features)
        features_df.reset_index(inplace=True)  # makes 'planet_name' a column

        write_header = not os.path.exists(simulated_params_filename)

        params_df.to_csv(simulated_params_filename, mode='a', header=write_header, index=False)
        features_df.to_csv(simulated_curve_features_filename, mode='a', header=write_header, index=False)

        # Clear batch memory
        simulated_params.clear()
        features.clear()

print(f"Finished Generating {i + 1} lightcurves")

/content/drive/MyDrive/Berkeley_AIML/Capstone/lightcurves/orbital_prediction_simcurve_XGB_model1/simcurves_fitted_params2.csv
/content/drive/MyDrive/Berkeley_AIML/Capstone/lightcurves/orbital_prediction_simcurve_XGB_model1/simcurves_fitted_features2.csv
Generating lightcurves...
Processing batch ending at simulation 200
Processing batch ending at simulation 400
Processing batch ending at simulation 600
Processing batch ending at simulation 800
Processing batch ending at simulation 1000
Processing batch ending at simulation 1200
Processing batch ending at simulation 1400
Processing batch ending at simulation 1600
Processing batch ending at simulation 1800
Processing batch ending at simulation 2000
Processing batch ending at simulation 2200
Processing batch ending at simulation 2400
Processing batch ending at simulation 2600
Processing batch ending at simulation 2800
Processing batch ending at simulation 3000
Processing batch ending at simulation 3200
Processing batch ending at simulatio

In [ ]:
#Perform Feature Extraction for Simulated Curves
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tqdm import tqdm
import os

# === CONFIGURATION ===

INPUT_FILE =  os.path.join(OUTPUT_PATH, "consolidated_sim_curves.csv")   # should contain columns: id, time, value
OUTPUT_FILE  = os.path.join(OUTPUT_PATH, "simulated_features_tsfresh.csv")
CHUNK_SIZE = 1                                             # 1 ID at a time
SAVE_EVERY = 100                                           # Save intermediate results every 100 IDs (optional)
DISABLE_PROGRESSBAR = True

# === LOAD merged_df ===
print("Loading merged lightcurve data...")
merged_df = pd.read_csv(INPUT_FILE)

# === Clean 'value' column ===
print("Cleaning 1")
merged_df['value'] = pd.to_numeric(merged_df['value'], errors='coerce')
print("Cleaning 2")
merged_df = merged_df.dropna(subset=['value'])
print("Cleaning 3")
merged_df = merged_df[np.isfinite(merged_df['value'])]

# === Initialize final feature DataFrame ===
all_features = []

# === Loop over unique IDs and extract features one by one ===
ids = merged_df['id'].unique()

for idx, single_id in enumerate(tqdm(ids, desc="Extracting features", disable=DISABLE_PROGRESSBAR)):
    try:
        temp_df = merged_df[merged_df['id'] == single_id]

        # Skip short time series
        if len(temp_df) < 10:
            continue

        feats = extract_features(
            temp_df,
            column_id='id',
            column_sort='time',
            column_value='value',
            default_fc_parameters=EfficientFCParameters(),
            disable_progressbar=True,
            show_warnings=True,
            n_jobs=2
        )

        all_features.append(feats)

        # Optional: save every N chunks
        if (idx + 1) % SAVE_EVERY == 0:
            temp_result = pd.concat(all_features)
            temp_result.to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE))
            all_features = []  # Clear memory
            print(f"Saved intermediate results to {OUTPUT_FILE}, i= {i+1}")

    except Exception as e:
        print(f"❌ Failed for ID = {single_id}: {type(e).__name__} - {e}")

# === Final save ===
if all_features:
    final_result = pd.concat(all_features)
    final_result.to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE))

print(f"\n✅ Feature extraction complete. Features saved to: {OUTPUT_FILE}")


In [ ]:
###########################################
# EXTRACT FEATURES FOR SIMULATED LIGHTCURVES
###########################################

# Batch Recovery version
import os
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tqdm import tqdm

# === CONFIGURATION ===
print("Starting Recovery version of TSFRESH feature extraction....")
OUTPUT_FILE  = os.path.join(OUTPUT_PATH, "simulated_features_tsfresh.csv")
PROCESSED_PLANETS_FILE  = os.path.join(OUTPUT_PATH, "processed_planets.csv")
BATCH_SIZE = 50
DISABLE_PROGRESSBAR = False
print("Paths created")
# === Load already-processed planet names ===
if os.path.exists(PROCESSED_PLANETS_FILE):
    print(f"Loading existing features from {PROCESSED_PLANETS_FILE}...")
    #processed_df = pd.read_csv(PROCESSED_PLANETS_FILE, index_col=0)
    processed_df = pd.read_csv(PROCESSED_PLANETS_FILE)
    # Get unique planet names from the index
    processed_planets = set(processed_df['planet_name'])
    print(f"Already processed: {len(processed_planets)} planet names.")
else:
    processed_planets = set()
print("Sorting Filenames........")
# === Gather all input files ===
all_files = sorted([
    f for f in os.listdir(SIMULATED_CURVES_PATH)
    if f.endswith('_simcurve.csv')
])

print(f"Found {len(all_files)} total files.")

# === Batch collector ===
batch_features = []
batch_count = 0
failed_files = []
i=0
# === Loop through each file ===
for file in tqdm(all_files, desc="Processing", disable=DISABLE_PROGRESSBAR):
    try:
        # Derive planet name from filename
        planet_name = file.replace('_simcurve.csv', '').replace('_', ' ')
        i= i+ 1
        print(f"Processing File {i}: {planet_name}")
        if planet_name in processed_planets:
            continue  # Already processed

        # Load and validate the file
        df = pd.read_csv(os.path.join(SIMULATED_CURVES_PATH, file))
        if 'flux' in df.columns:
            df = df.rename(columns={'flux': 'value'})

        if not {'planet_name', 'time', 'value'}.issubset(df.columns):
            print(f"⚠️ Skipping {file}: missing required columns.")
            continue

        #df['value'] = pd.to_numeric(df['value'], errors='coerce')
        #df = df.dropna(subset=['value'])
        #df = df[np.isfinite(df['value'])]

        if len(df) < 10:
            continue

        # Extract features
        feats = extract_features(
            df,
            column_id='planet_name',
            column_sort='time',
            column_value='value',
            default_fc_parameters=EfficientFCParameters(),
            disable_progressbar=True,
            show_warnings=True,
            n_jobs=0
        )

        if feats.empty:
            continue

        # Add planet name to the result
        feats['planet_name'] = planet_name
        feats.set_index('planet_name', inplace=True, append=False)

        batch_features.append(feats)
        batch_count += 1

        # Save batch if ready
        if batch_count >= BATCH_SIZE:
            combined = pd.concat(batch_features)
            if not os.path.exists(OUTPUT_FILE):
                combined.to_csv(OUTPUT_FILE)
            else:
                combined.to_csv(OUTPUT_FILE, mode='a', header=False)
            print(f"✅ Saved batch of {batch_count} features to {OUTPUT_FILE}")
            batch_features = []
            batch_count = 0
            processed_planets.update(combined.index)

    except Exception as e:
        print(f"❌ Failed on file {file}: {type(e).__name__} - {e}")
        failed_files.append(file)

# === Final batch save ===
if batch_features:
    combined = pd.concat(batch_features)
    if not os.path.exists(OUTPUT_FILE):
        combined.to_csv(OUTPUT_FILE)
    else:
        combined.to_csv(OUTPUT_FILE, mode='a', header=False)
    print(f"✅ Final batch saved ({len(batch_features)} entries).")

# === Log failed files (optional) ===
if failed_files:
    with open("failed_simcurve_files.txt", "w") as f:
        for fname in failed_files:
            f.write(fname + '\n')

print("\n🏁 All done.")
